In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Definizione del problema

Il dataset in esame (https://www.kaggle.com/datasets/ananthr1/weather-prediction/data) contiene dati sulle condizioni meteorologiche di Seattle dal 01/01/2012 al 31/12/2015.
L'obiettivo è quello di prevedere le condizioni meteo (es. sole, pioggia, nebbia) di un determinato giorno, si tratta quindi di un problema di classificazione.                                   

In [3]:
dataset = pd.read_csv("seattle-weather.csv")

# Analisi del dataset

In [4]:
dataset.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


Descrizione delle feature:
- date -> data di riferimento
- precipitation -> valore numerico che indica l'intensità delle precipitazioni in qualunque forma (pioggia, neve, ...)
- temp_max -> temperatura massima giornaliera
- temp_min -> temperaura minima giornaliera
- wind -> intensità del vento
- weather -> etichetta che indica le condizioni meteorologiche della giornata

In questo caso la feature target è "weather", ossia si vuole prevedere le condizioni meteorologiche di una certa giornata.

Almeno inizialmente saranno prese in considerazione tutte le altre feature ai fini della predizione.

Si vuole ora verificare che sia effettivamente presente una riga per ogni data nell'intervallo. 

In [ ]:
# Si crea una serie contentente tutte le date presenti del dataset
dates = pd.to_datetime(dataset['date'])

# Si crea l'intervallo completo tra la data minima e massima
date_range = pd.date_range(start=dates.min(), end=dates.max(), freq='D')

# Si crea una serie con le date mancanti e si verifica che sia vuota
missing_dates = date_range.difference(dates)
print(missing_dates.empty)


True


Tutte le date nell'intervallo risultano presenti, questo rende il dataset comodamente utilizzabile come serie temporale.